# Ch 18 State Space Models


## 18.3.2 Kalman smoothing algorithm

* kalman filter
  * p(z_t|y_1:t), no future data, online like tracking
* kalman smoothing
  * p(z_t|y_1:T), with future data, offline
  * 미래 데이터 활용으로 인해 불확실성 감소
    * 아래 (c) 처럼 neighbor가 많은 부분은 불확실성이 감소
    * 시작과 끝부분은 상대적으로 neighbor가 적어서 감소 효과가 적다

![](./images/ch18/19.png) 


### notation recall

![](./images/ch18/21.png) 


### algorithm

* HMM의 forward-backward 알고리즘과 유사
  * 즉 UGM에서의 message passing의 원리
  * 왼쪽에서 시작해서 오른쪽으로 그래프 끝까지 전파, p(z_T|y_1:T)
  * 이를 다시 거꾸로 역전파(future back to the past)
  * 정방향 정보와 역방향 정보를 결합
* HMM의 forward-backward 와의 차이점 
  * HMM의 backward는 forward를 선행하지 않고 단독으로 진행 가능
    * 그 과정중 observation data 필요
  * kalman smoothing에서는 forward가 반드시 선행되야 함. 
    * 반면 observation data 필요없고, 정방향 정보만 필요

![](./images/ch18/20.png) 

### derivation - skip


## 18.4 Learning for LG-SSM

* how to estimate parameters of of LG-SSM, namely, A,C,Q,R below
  * in control theory, called system identification

![](./images/ch18/02.png)

* special case - well known A, C 
  * ex) time series forcasting, or physical state estimation
  * 단지 Q와 R만 추정하면 된다. 

### Identifiability and numerical stability

* without loss of generality
  * assume Q = I => noise 는 A로 적절히 표현 가능 
  * assume R to be diagonal => reduce DOF => improve numerical stability
* eigenvalue of A
  * if no system noise, and eigenvalue greater than 1 => blow up
  * all eigenvalues be less than 1 with non-zero noise

![](./images/ch18/22.png)

### EM for LG-SSM

* HMM 의 Baum-welch 와 유사, 
  * 참고 : https://web.stanford.edu/~lmackey/stats306b/doc/stats306b-spring14-lecture11_scribed.pdf
  
![](./images/ch18/23.png)
![](./images/ch18/24.png)
![](./images/ch18/25.png)


### Subspace methods for LG-SSM

* EM 방식의 단점
  * 초기 parameter 추정
* Subspace methods for system identification 
 * project y onto subspace of z, and using PCA, identify subspace
 * https://www.youtube.com/watch?v=EMdrHvYd_Zs 
 * https://simsee.org/simsee/biblioteca/Springer,%20Subspace%20Methods%20For%20System%20Identification%20(2005)%20Ddu%20Lotb.pdf 

### Baysian methods for LG-SSM

* offline baysian alternatives
  * Variational Bayes EM (Beal 2003)
  * Blocked Gibbs Sampling (Carter 1994)


## 18.5 Approximate online inference for non-linearm non-gaussian SSMs

* 지금까지는 easy-case
  * linear and gauusian model
  * gaussian noise
* 일반적으로는 
  * non-linear, non-gaussian model
  * non-gaussian noise
  

### 18.5.1 Extended Kalman filter(EKF)

* 약간 어려운 case
  * non-liear model
  * gassuian nose
* approximate inference
  * approximate the posterior by a Gaussian
  * Y=f(X)
    * X be gaussian
    * f be non-linear function
    * Y be approximate gaussian

* 기본 아이디어
  * 각 시점에서 선형 근사화(first-order taylor approx)
  * 그런다음 standard kalman filter 적용

![](./images/ch18/26.png)  
![](./images/ch18/27.png)

![](./images/ch18/28.png)
![](./images/ch18/29.png)
![](./images/ch18/30.png)

* 잘 작동하지 않는 경우 
  * prior covariancerk 너무 펑퍼짐할 때, 초기부터 많은 probability mass를 linearized 과정 중에 유실
  * mapping function이 nonlinear near the current mean 일 때 

  


### 18.5.2. unscented kalman filter(UKF)

In [ ]:
### 18.5.3. Assumed density filtering (ADF)

In [ ]:
## 18.6 Hybrid discrete/continuous SSMs